# Poker Rule Induction


## Keras - 2 Attempts

## Attempt 1 

In [2]:

import random
import os
import time
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# 1. Reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)

# 2. Data Loading
train_df = pd.read_csv('poker-rule-induction/train.csv.zip', compression='zip')
df_test = pd.read_csv('poker-rule-induction/test.csv.zip', compression='zip')
submission_example = pd.read_csv('poker-rule-induction/sampleSubmission.csv.zip', compression='zip')

id_col = submission_example.columns[0]
target_columns = submission_example.columns.tolist()[1:]

df = pd.concat([train_df], ignore_index=True)

# 3. Target Encoding
le = LabelEncoder().fit(df[target_columns[0]].astype(str))
y_enc = le.transform(df[target_columns[0]].astype(str))
num_classes = len(le.classes_)

# 4. Train/Test Split
X = df.drop(columns=target_columns + [id_col], errors='ignore')
train_ids = df[id_col]
test_ids = df_test[id_col]
X_train = X.copy()
y_train = y_enc
X_val = df_test.drop(columns=target_columns + [id_col], errors='ignore')
y_val = None

# 5. Feature Engineering
# Drop columns with all missing values
X_train = X_train.loc[:, X_train.notnull().any()]
X_val = X_val[X_train.columns]

# Identify numeric and categorical
numeric_cols = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object','category']).columns.tolist()

# 6. Preprocessing Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

input_dim = X_train_proc.shape[1]

# 7. Model Architecture for small feature set
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(min(input_dim*2,128), activation='relu', input_shape=(input_dim,)))
model.add(Dropout(0.2))
model.add(Dense(min(input_dim,64), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# 8. Compile
top_k = min(num_classes,5)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseTopKCategoricalAccuracy(k=top_k, name=f'top_{top_k}_accuracy')]
)

# 9. Callbacks & Training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('model.h5', save_best_only=True, monitor='val_loss')
]
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# 10. Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 11. Prediction & Submission
raw_preds = model.predict(X_val_proc)
idxs = raw_preds.argmax(axis=1)
final = le.inverse_transform(idxs)
final = final.reshape(-1,1)

submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)


KeyError: 'id'

# Attempt 2

In [3]:
import random
import os
import time
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# 1. Reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)

# 2. Data Loading
train_df = pd.read_csv('poker-rule-induction/train.csv.zip', compression='zip')
df_test = pd.read_csv('poker-rule-induction/test.csv.zip', compression='zip')
submission_example = pd.read_csv('poker-rule-induction/sampleSubmission.csv.zip', compression='zip')

id_col = submission_example.columns[0]
target_columns = submission_example.columns.tolist()[1:]

df = pd.concat([train_df], ignore_index=True)

# 3. Target Encoding
le = LabelEncoder().fit(df[target_columns[0]].astype(str))
y_enc = le.transform(df[target_columns[0]].astype(str))
num_classes = len(le.classes_)

# 4. Train/Test Split
X = df.drop(columns=target_columns + [id_col], errors='ignore')
train_ids = df[id_col] if id_col in df.columns else None  # Fixing the KeyError
test_ids = df_test[id_col] if id_col in df_test.columns else None  # Fixing the KeyError
X_train = X.copy()
y_train = y_enc
X_val = df_test.drop(columns=target_columns + [id_col], errors='ignore')
y_val = None

# 5. Feature Engineering
# Drop columns with all missing values
X_train = X_train.loc[:, X_train.notnull().any()]
X_val = X_val[X_train.columns]

# Identify numeric and categorical
numeric_cols = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object','category']).columns.tolist()

# 6. Preprocessing Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

input_dim = X_train_proc.shape[1]

# 7. Model Architecture for small feature set
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(min(input_dim*2,128), activation='relu', input_shape=(input_dim,)))
model.add(Dropout(0.2))
model.add(Dense(min(input_dim,64), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# 8. Compile
top_k = min(num_classes,5)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseTopKCategoricalAccuracy(k=top_k, name=f'top_{top_k}_accuracy')]
)

# 9. Callbacks & Training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('model.h5', save_best_only=True, monitor='val_loss')
]
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# 10. Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 11. Prediction & Submission
raw_preds = model.predict(X_val_proc)
idxs = raw_preds.argmax(axis=1)
final = le.inverse_transform(idxs)
final = final.reshape(-1,1)

submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Epoch 1/100


/home/exouser/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-09 23:03:27.018224: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


313/313 - 2s - 5ms/step - accuracy: 0.4576 - loss: 1.3832 - top_5_accuracy: 0.9577 - val_accuracy: 0.5002 - val_loss: 1.0134 - val_top_5_accuracy: 0.9914
Epoch 2/100


313/313 - 1s - 2ms/step - accuracy: 0.4792 - loss: 1.0887 - top_5_accuracy: 0.9911 - val_accuracy: 0.5100 - val_loss: 0.9874 - val_top_5_accuracy: 0.9914
Epoch 3/100


313/313 - 1s - 2ms/step - accuracy: 0.4814 - loss: 1.0453 - top_5_accuracy: 0.9932 - val_accuracy: 0.5156 - val_loss: 0.9817 - val_top_5_accuracy: 0.9928
Epoch 4/100


...

313/313 - 1s - 2ms/step - accuracy: 0.5445 - loss: 0.9447 - top_5_accuracy: 0.9975 - val_accuracy: 0.5664 - val_loss: 0.9352 - val_top_5_accuracy: 0.9966
Epoch 80/100
313/313 - 1s - 2ms/step - accuracy: 0.5457 - loss: 0.9460 - top_5_accuracy: 0.9974 - val_accuracy: 0.5702 - val_loss: 0.9355 - val_top_5_accuracy: 0.9966
Epoch 81/100
313/313 - 1s - 2ms/step - accuracy: 0.5476 - loss: 0.9463 - top_5_accuracy: 0.9974 - val_accuracy: 0.5692 - val_loss: 0.9353 - val_top_5_accuracy: 0.9966
Epoch 82/100
313/313 - 1s - 2ms/step - accuracy: 0.5452 - loss: 0.9426 - top_5_accuracy: 0.9974 - val_accuracy: 0.5688 - val_loss: 0.9360 - val_top_5_accuracy: 0.9966
Epoch 83/100
313/313 - 1s - 2ms/step - accuracy: 0.5441 - loss: 0.9462 - top_5_accuracy: 0.9978 - val_accuracy: 0.5706 - val_loss: 0.9356 - val_top_5_accuracy: 0.9966
Epoch 84/100
313/313 - 1s - 2ms/step - accuracy: 0.5459 - loss: 0.9467 - top_5_accuracy: 0.9976 - val_accuracy: 0.5684 - val_loss: 0.9357 - val_top_5_accuracy: 0.9966
Epoch 85/10

313/313 - 1s - 2ms/step - accuracy: 0.5482 - loss: 0.9436 - top_5_accuracy: 0.9974 - val_accuracy: 0.5710 - val_loss: 0.9352 - val_top_5_accuracy: 0.9968
Epoch 87/100


313/313 - 1s - 2ms/step - accuracy: 0.5444 - loss: 0.9446 - top_5_accuracy: 0.9978 - val_accuracy: 0.5718 - val_loss: 0.9346 - val_top_5_accuracy: 0.9968
Epoch 88/100
313/313 - 1s - 2ms/step - accuracy: 0.5442 - loss: 0.9453 - top_5_accuracy: 0.9976 - val_accuracy: 0.5702 - val_loss: 0.9349 - val_top_5_accuracy: 0.9968
Epoch 89/100


313/313 - 1s - 2ms/step - accuracy: 0.5471 - loss: 0.9442 - top_5_accuracy: 0.9977 - val_accuracy: 0.5684 - val_loss: 0.9346 - val_top_5_accuracy: 0.9968
Epoch 90/100
313/313 - 1s - 2ms/step - accuracy: 0.5456 - loss: 0.9442 - top_5_accuracy: 0.9973 - val_accuracy: 0.5690 - val_loss: 0.9354 - val_top_5_accuracy: 0.9968
Epoch 91/100
313/313 - 1s - 2ms/step - accuracy: 0.5453 - loss: 0.9435 - top_5_accuracy: 0.9978 - val_accuracy: 0.5684 - val_loss: 0.9348 - val_top_5_accuracy: 0.9968
Epoch 92/100


313/313 - 1s - 2ms/step - accuracy: 0.5479 - loss: 0.9423 - top_5_accuracy: 0.9977 - val_accuracy: 0.5682 - val_loss: 0.9342 - val_top_5_accuracy: 0.9966
Epoch 93/100
313/313 - 1s - 2ms/step - accuracy: 0.5412 - loss: 0.9445 - top_5_accuracy: 0.9976 - val_accuracy: 0.5664 - val_loss: 0.9347 - val_top_5_accuracy: 0.9966
Epoch 94/100
313/313 - 1s - 2ms/step - accuracy: 0.5475 - loss: 0.9450 - top_5_accuracy: 0.9975 - val_accuracy: 0.5700 - val_loss: 0.9353 - val_top_5_accuracy: 0.9966
Epoch 95/100
313/313 - 1s - 2ms/step - accuracy: 0.5463 - loss: 0.9424 - top_5_accuracy: 0.9975 - val_accuracy: 0.5706 - val_loss: 0.9347 - val_top_5_accuracy: 0.9966
Epoch 96/100
313/313 - 1s - 2ms/step - accuracy: 0.5439 - loss: 0.9450 - top_5_accuracy: 0.9976 - val_accuracy: 0.5718 - val_loss: 0.9348 - val_top_5_accuracy: 0.9966
Epoch 97/100
313/313 - 1s - 2ms/step - accuracy: 0.5426 - loss: 0.9431 - top_5_accuracy: 0.9976 - val_accuracy: 0.5730 - val_loss: 0.9348 - val_top_5_accuracy: 0.9966
Epoch 98/10

In [ ]:
print(duration)

## Keras Tuner - 1 Attempt

## Attempt 1


In [ ]:
import random
import os
import time
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# 1. Reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)

# 2. Data Loading
train_df = pd.read_csv('train.csv.zip', compression='zip')
df_test = pd.read_csv('test.csv.zip', compression='zip')
submission_example = pd.read_csv('sampleSubmission.csv.zip', compression='zip')

id_col = submission_example.columns[0]
target_columns = submission_example.columns.tolist()[1:]

df = pd.concat([train_df], ignore_index=True)

# 3. Target Encoding
le = LabelEncoder().fit(df[target_columns[0]].astype(str))
y_enc = le.transform(df[target_columns[0]].astype(str))
num_classes = len(le.classes_)

# 4. Train/Test Split
X = df.drop(columns=target_columns + [id_col], errors='ignore')
train_ids = df[id_col] if id_col in df.columns else None  # Fixing the KeyError
test_ids = df_test[id_col] if id_col in df_test.columns else None  # Fixing the KeyError
X_train = X.copy()
y_train = y_enc
X_val = df_test.drop(columns=target_columns + [id_col], errors='ignore')
y_val = None

# 5. Feature Engineering
# Drop columns with all missing values
X_train = X_train.loc[:, X_train.notnull().any()]
X_val = X_val[X_train.columns]

# Identify numeric and categorical
numeric_cols = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object','category']).columns.tolist()

# 6. Preprocessing Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

input_dim = X_train_proc.shape[1]

# 7. Model Architecture for small feature set
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Input dimension
n_features = input_dim

# HyperModel
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        outputs = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

# Tuner
bs = 32  # batch size
ep = 20  # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

tuner.search(
    X_train_proc, y_train,
    validation_split=0.2,
    batch_size=bs, epochs=ep,
    callbacks=[early_stopping, checkpoint]
)

model = tuner.hypermodel.build(
    tuner.get_best_hyperparameters(1)[0]
)

start_time = time.time()  # Start timing


# Retrain model with original callbacks and data
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100, batch_size=bs,
    callbacks=[early_stopping, checkpoint],
    verbose=2
)

duration = time.time() - start_time  # Calculate duration


# 10. Evaluation & Logging
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 11. Prediction & Submission
raw_preds = model.predict(X_val_proc)
idxs = raw_preds.argmax(axis=1)
final = le.inverse_transform(idxs)
final = final.reshape(-1,1)

submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 01m 17s]
val_loss: 0.9246384501457214

Best val_loss So Far: 0.9138667583465576
Total elapsed time: 00h 13m 54s
Epoch 1/100


626/626 - 9s - 14ms/step - accuracy: 0.5146 - loss: 0.9990 - val_accuracy: 0.5426 - val_loss: 0.9683
Epoch 2/100


626/626 - 2s - 4ms/step - accuracy: 0.5358 - loss: 0.9614 - val_accuracy: 0.5428 - val_loss: 0.9605
Epoch 3/100


626/626 - 2s - 4ms/step - accuracy: 0.5398 - loss: 0.9535 - val_accuracy: 0.5472 - val_loss: 0.9595
Epoch 4/100


626/626 - 2s - 4ms/step - accuracy: 0.5423 - loss: 0.9486 - val_accuracy: 0.5434 - val_loss: 0.9566
Epoch 5/100


...

626/626 - 2s - 3ms/step - accuracy: 0.8490 - loss: 0.3963 - val_accuracy: 0.8603 - val_loss: 0.4687
Epoch 95/100
626/626 - 2s - 3ms/step - accuracy: 0.8525 - loss: 0.3877 - val_accuracy: 0.8563 - val_loss: 0.4717
Epoch 96/100
626/626 - 2s - 4ms/step - accuracy: 0.8555 - loss: 0.3834 - val_accuracy: 0.8581 - val_loss: 0.4725
Epoch 97/100


626/626 - 2s - 4ms/step - accuracy: 0.8552 - loss: 0.3757 - val_accuracy: 0.8657 - val_loss: 0.4618
Epoch 98/100


626/626 - 2s - 3ms/step - accuracy: 0.8567 - loss: 0.3782 - val_accuracy: 0.8687 - val_loss: 0.4531
Epoch 99/100


626/626 - 2s - 3ms/step - accuracy: 0.8598 - loss: 0.3738 - val_accuracy: 0.8749 - val_loss: 0.4393
Epoch 100/100


626/626 - 2s - 4ms/step - accuracy: 0.8638 - loss: 0.3665 - val_accuracy: 0.8766 - val_loss: 0.4343
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 40s 1ms/step


In [ ]:
print(duration)

249.279543
